In [1]:
import os,glob,sys
import xarray as xr
import xesmf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
import dask
from direct_effect_analysis import DirectEffectAnalysis
sys.path.append('../')
from _decomp import *
from linear_regression import get_slope_and_pval


In [2]:
# Define summer months (June, July, August)
target_variable='TREFHT'
cov_variable = 'Z500'
months=[6,7,8]
period = [1979,2023]
run = 'ERA5'
gmst_rolling_window_size_in_days = 30

In [3]:
oo = decomp_ERA5(target_variable=target_variable, months=months, period=period)
oo.target_open(0, 360, 30, 70, var_name_in_file='var167', cdo_options='-remapcon,r360x180')
target = oo._data_raw['target']

In [4]:
cdo_input_string = oo.get_cdo_input_string(var='TREFHT')
tmp_file = cdo.fldmean(input=f"-selyear,{period[0]}/{period[1]} {cdo_input_string}", output=f"{cdo_tmp_dir}/GMT_daily_{period[0]}-{period[1]}.nc")
n_days = 30
gmst = xr.load_dataarray(tmp_file).squeeze()
gmst = gmst.rolling(time=n_days, center=True).mean()
gmst = gmst.sel(time=gmst['time.month'].isin(months)).data[:, None]

In [5]:
cdo_input_string = oo.get_cdo_input_string(var=cov_variable)
lon1, lon2, lat1, lat2 = 0, 360, -12, 90
with open(f"{cdo_tmp_dir}/grid_6x6_{lat1}Nto{lat2}N.txt", 'w') as fl:
    fl.write(f'''
gridtype  = lonlat
xsize     = 60
ysize     = 17
xfirst    = 0
xinc      = 6
yfirst    = {lat1}
yinc      = 6    
''')
tmp_file = cdo.remapcon(f"{cdo_tmp_dir}/grid_6x6_{lat1}Nto{lat2}N.txt", 
        input=f"-selmon,{','.join([str(m) for m in months])} {cdo_input_string}", 
        output=f"{cdo_tmp_dir}/{cov_variable}_{oo._data_tag}_{lon1},{lon2},{lat1},{lat2}_6x6.nc")
        
z500 = xr.load_dataarray(tmp_file).squeeze()
z500_global = xr.load_dataarray(f'{cdo_tmp_dir}/var129_ERA5_E5_6m7m8_GL.nc').squeeze()
z500 = z500 - z500_global
z500 = z500.loc['1979':'2023']

In [6]:
# remove seasonality
z500 = (z500.groupby('time.month') - z500.groupby('time.month').mean())
target = (target.groupby('time.month') - target.groupby('time.month').mean())

gmst = gmst - gmst.mean()

# Saving lat, lon and time
lats = target.lat.data
lons = target.lon.data
time = target.time.data

# Converting xarray to numpy array of correct dimensions
X_2d = z500.values.reshape((len(time), -1))
Y_2d = target.values.reshape((len(time), -1))

# training
X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X_2d, Y_2d, gmst, test_size=0.2)
n_cps = np.logspace(0.15, 2.2, 20).astype('int')
dea = DirectEffectAnalysis(n_components='optimal', n_cps=n_cps, k_fold=5)
dea.fit(X_train, Y_train, Z_train, fit_test=False)

100%|██████████| 20/20 [01:20<00:00,  4.05s/it]


(None, None)

In [7]:
Y_dyn, Y_thermo = dea.counterfactual(Y_2d, gmst)

In [8]:
def transform_to_xarray(Y_):
    Y_ = xr.DataArray(
    Y_.reshape((-1, len(lats), len(lons))),
        dims = target.dims,
        coords = target.coords)
    Y_ = Y_.assign_coords(year=Y_.time.dt.year)
    Y_ = Y_.swap_dims({'time': 'year'})
    Y_ = Y_.groupby('year').mean()
    return Y_

In [10]:
slope, pval = get_slope_and_pval(transform_to_xarray(Y_dyn))
xr.Dataset(
    {'slope':slope, 'pval':pval}
).to_netcdf(f'/climca/people/ppfleiderer/decomposition/DEA_homer/ERA5_trend_1979-2023_circ.nc')

In [11]:
slope, pval = get_slope_and_pval(transform_to_xarray(Y_thermo))
xr.Dataset(
    {'slope':slope, 'pval':pval}
).to_netcdf(f'/climca/people/ppfleiderer/decomposition/DEA_homer/ERA5_trend_1979-2023_thermo.nc')